## Example 3: Hessian and Normal Modes
The luminous efficiency is defined by the following integral:
\begin{equation}
\eta_{lum} = \frac{\int_0^{\infty} BB(\lambda, T) \epsilon(\lambda) v(\lambda) d\lambda}{\int_0^{\infty} BB(\lambda, T) \epsilon(\lambda)d\lambda},
\end{equation}
where $BB(\lambda, T)$ is Planck's blackbody law, $\epsilon(\lambda)$ is the emissivity function 
of the structure, and $v(\lambda)$ is the photopic luminosity function that characterizes the color sensisitivity of the human eye.  Only the $\epsilon(\lambda)$ function depends on the structure itself.

One model structure we have examined for related selective emitter applications (i.e. solar thermophotovoltaics) is the "critically-coupled" selective emitter.  Such an emitter may include both a Bragg reflector component (alternating transparent materials like glass and titania) and a weak-absorber (which may be an alloy of a metal and a transparent material, or may take on some other form).  The point of the former component is to strongly reflect light across a range of wavelengths, and the point of the latter is to absorb light across a range of wavelengths.  When the absorption and reflection ranges are well-mateched, critically coupling can be achieved whereby the absorbing layer ultimately absorbs almost all incident light in a limited wavelength range.  This property also gives the structure the ability to thermally emit light in that same wavelength range.  This model is promising because the reflection wavelength range can be tuned fairly easily in Bragg reflectors by choice of layer thickness and choice
of contrast between the low- and high-RI layers.  

In particular, the center wavelength of the Bragg Reflectors strong reflection is approximately
\begin{equation}
\lambda_c = 4 \cdot n \cdot d,
\end{equation}
where $n$ is the refractive index of the layer and $d$ is the thickness of the layer.  Sometimes, people report central frequency rather than central wavelength for a Bragg Reflector, and the central frequency is related by 
\begin{equation}
f_c = \frac{c}{\lambda_c}.
\end{equation}
The frequency range around the central frequency is related to the difference between the high ($n_2$) and low ($n_1)$ refractive index layers as
\begin{equation}
\Delta f = \frac{4}{\pi} {\rm arcsin}\left( \frac{n_2 - n_1}{n_2 + n_1}\right) \cdot f_c.
\end{equation}

In other words, the features of the reflectance spectrum of the Bragg Reflector can be tuned through choice of materials and thicknesses of the Bragg Reflector layers.  A simple hypothesis would be that an ideal Bragg Reflector component for the Incandescent bulb structure would be one whose central frequency and frequency width align perfectly with the photopic luminosity function.  

### Question 1: What should $f_c$ be for a Bragg Reflector which is perfectly matched for the photopic luminosity function

### Question 2: Assuming $n_2 = 2.4$ (similar to TiO$_2$), what should $n_1$ so that the bandwidth of this Bragg reflector matches the bandwidth of the photopic luminosity function?

### Question 3: What is an actual material with a refractive index close to the $n_1$ you found above?

### Question 4: What should $d_1$ and $d_2$ be such that the Bragg reflector has the desired center wavelength to match the photopic luminosity function?  ($d_1$ is the thickness of the layer that has refractive index $n_1$, etc).

The second component is the weak absorber component.  Perhaps the simplest model for a tunable weak absorber is a material whose refractive index is approximated by the so-called Lorentz Oscillator model:
\begin{equation}
n_L(\omega) = \sqrt{1 + \frac{\omega_P^2}{\omega_0^2 - \omega^2 - i \gamma \omega} },
\end{equation}
where $\omega = 2\pi \cdot f$ is the angular frequency, $\omega_P$ is called the bulk Plasmon frequency of the material, $\omega_0$ is the central angular frequency of absorption, $\gamma$ is related to the bandwidth around the central frequency (larger $\gamma$ means a broader absorption peak, smaller $\gamma$ means a narrower absorption peak).

WPTherml has a built-in function called *layer_lorentz(layer, omega_p, omega_0, gamma)* that defines 
the *layer*-th layer in a multi-layer structure to have a refractive index specified by the Lorentz oscillator function defined with parameters $\omega_P$ = *omega_p*, $\omega_0$ = *omega_0*, and $\gamma$ = *gamma*.  The following code plots the visible and near IR absorptivity/emissivity of a 10 nm slab 
with refractive index given by the Lorentz oscillator with 
$\omega_0 = \frac{2 \pi c}{530 \cdot 10^{-9} m}, \gamma = \frac{2 \pi c}{7000 \cdot 10^{-9} m},$
and $\omega_P = \frac{2 \pi c}{1000 \cdot 10^{-9} m}$ where $c = 299792458 m/s$ (the speed of light).


In [2]:
from wptherml.wpml import multilayer
from wptherml.datalib import datalib
from matplotlib import pyplot as plt
import numpy as np
from scipy.interpolate import UnivariateSpline


### dictionary that stores basic properties 
### of the multilayer structure you want to simulate
structure = {
        ### No material actually called "Lorentz", so just use SiO2 as a placeholder
        ### that we can change later
        'Material_List' : ['Air', 'SiO2', 'Air'], 
        'Thickness_List': [0, 10e-9, 0],
        'Lambda_List': [300e-9, 4000e-9, 1000],
        'Temperature': 2700,
        'LIGHTBULB': 1
     
        }


### create instance of multilayer
lorentz_slab = multilayer(structure)

### define Lorentz parameters
c = 299792458
layer = 1
omega_0 = 2*np.pi*c/800e-9
gamma = 2*np.pi*c/7000e-9
omega_p = 2*np.pi*c/1000e-9

### create a layer in the multilayer structure defined
### by those Lorentz parameters
lorentz_slab.layer_lorentz(layer, omega_p, omega_0, gamma)
### Update optical properties with new material
lorentz_slab.fresnel()
### Update thermal emission properties with new material
lorentz_slab.thermal_emission()
### Update luminous efficiency with new material
lorentz_slab.luminous_efficiency()
### Plot emissivity and print efficiency!
plt.plot(lorentz_slab.lambda_array*1e9, lorentz_slab.emissivity_array, 'blue')


plt.show()
print("Luminous Efficiency Value is ",100*lorentz_slab.luminous_efficiency_val)
y = lorentz_slab.emissivity_array
x = lorentz_slab.lambda_array
half_max = max(y)/2
f_lam = x[np.argmax(y)]
fc = c/f_lam

print('Central frequency is', fc*1e9,'nm')

##############################################################################
spline = UnivariateSpline(x, y-np.max(y)/2, s=0)
r1, r2 = spline.roots() # find the roots
delta_lam_ideal = r2-r1
delta_f_ideal = c/delta_lam_ideal
##################################################################################
n2 = 2.4
val = np.linspace(1, 2.4, 20) 

def delta_f_trial(n1):
    trial = 0
    min = 1000
    for i in range(0, len(val)):
        trial = (4/np.pi)*np.arcsin((n2-val[i])/(n2+val[i]))*fc
        if np.abs(trial-delta_f_ideal) <= min:
            min = np.abs(trial-delta_f_ideal)
            n1 = val[i]
        return n1
    
solve = delta_f_trial(val)
print(solve)
    

SyntaxError: invalid syntax (coolinglib.py, line 28)

### Question 5:  How should you choose $\omega_0$ and $\gamma$ to match the photopic luminosity function?  Plot the emissivity of a Lorentz oscillator slab that you believe is ideally suited for the photopic luminosity function.

### Question 6: What is the luminous efficiency of this Lorentz slab?

### Question 7: What happens when you combine the Lorentz oscillator slab with your previously-chosen Bragg reflector?  In particular, how does the emissivity profile change and how does the luminous efficiency change?  

To answer, create a new instance of the multi-layer class that has the Lorentz slab as its top-most layer and then has a Bragg reflector using materials chosen from Question 3, plot its emissivity spectrum and compute its Luminous efficiency.  Experiment a bit with the number of Bragg reflector layers; for example, try two repeats of alternating $n_1$ and $n_2$, three repeats, four repeats.  Note how the number of repeats changes the luminous efficiency and the spectra.

In [7]:
### Code to create new instance and compute relevant properties goes here!





One fundamental question is how best to change the various parameters of such a "critically-coupled" structure to improve its performance.  Analysis that might give insight takes inspiration from chemistry, where "normal modes" of a molecule are identified as the unique modes of motion, some of which are responsible for changing a molecule from one structure to another.  The normal modes for a molecule are obtained by first constructing the so-called Hessian matrix of the molecule (the second derivative of energy with respect to all possible changes in the atom positions), then identifying the eigenvalues and eigenvectors.  The eigenvectors represent the normal modes of motion and the eigenvalues represent their frequency (related to their energy).  Incidentally, IR spectroscopy probes the frequency/energy of such normal modes.

In our system, we can define the elements of the Hessian matrix as follows:
\begin{equation}
H_{ij} = \frac{\partial^2 \eta_{lum}}{\partial x_i \partial x_j},
\end{equation}
here $\eta_{lum}$ represents the luminous efficiency and $x_i$ and $x_j$ are 
different structural parameters that can be used to change the emissivity and the luminous efficiency 
of the structure.  For the critcally-coupled structures, these parameters include $\omega_P$, $\omega_0$, $\gamma$, $n_1$, $n_2$, $d_L$, $d_1$, and $d_2$; hence the Hessian is a 8x8 matrix!  In the following,
let's denote $x_1 = \omega_P, x_2 = \omega_0, x_3 = \gamma, x_4 = d_L, x_5 = n_1, x_6 = n_2, x_7 = d_1, x_8 = d_2$. 

The method of finite-differences can be used to compute the elements of the Hessian matrix.  For concreteness,
consider the element $H_{12}$:
\begin{equation}
\frac{\partial^2 \eta_{lum}(x_1, x_2, ..., x_8)}{\partial x_1 \partial x_2} = 
\frac{ \frac{\partial \eta_{lum}(x_1+h_1, x_2, ..., x_8) }{\partial x_2} - \frac{\partial \eta_{lum}(x_1 - h_1, x_2, ...,x_8)}{\partial x_2} }{2h_1},
\end{equation}
where
\begin{equation}
\frac{\partial \eta_{lum}(x_1+h_1, x_2, ..., x_8) }{\partial x_2} = 
\frac{\eta_{lum}(x_1, x_2+h_2, ..., x_8) - \eta_{lum}(x_1, x_2-h_2, ..., x_8) }{2h_2},
\end{equation}
and $h_1$ and $h_2$ are suitably small changes that can be made to the parameters $x_1$ and $x_2$, respectively.
The (somewhat cumbersome) function below computes the elements of the Hessian matrix for a system that includes
a Lorentz layer with thickness $d_L$ as the top-most layer, 2 periods of a Bragg-reflector with $n_1$, $d_1$ for the low-index layer and $n_2$, $d_2$ for the high-index layer:


In [3]:
### Re-define structure:
structure = {
        ### No material actually called "Lorentz", so just use SiO2 as a placeholder
        ### that we can change later
        'Material_List' : ['Air', 'SiO2', 'SiO2', 'TiO2', 'SiO2', 'TiO2', 'Air'], 
        ### Thicknesses just chosen arbitrarily, replace with "optimal" values
        'Thickness_List': [0, 10e-9, 150e-9, 100e-9, 150e-9, 100e-9,  0],
        'Lambda_List': [300e-9, 4000e-9, 1000],
        'Temperature': 2700,
        'LIGHTBULB': 1
     
        }

### define instance called 'cc'
cc = multilayer(structure)

### a function to take an array of values x_1, ..., x_8 as defined above,
### re-define the multi-layer structure according to its values, and
### then update the luminous efficiency accordingly:
def update_multilayer(x):
    ### recall x_1 is omega_p, x_2 is omega_0, x_3 is gamma
    cc.layer_lorentz(1, x[0], x[1], x[2])
    ### x_4 is d_L, thickness of Lorentz layer
    cc.d[1] = x[3]
    ### x_5 is n_1, which applies to layers 2 and 4:
    cc.layer_static_ri(2, x[4])
    cc.layer_static_ri(4, x[4])
    ### x_6 is n_2, hich applies to layers 3 and 5:
    cc.layer_static_ri(3, x[5])
    cc.layer_static_ri(5, x[5])
    ### x_7 is d_1, which applies to layers 2 and 4
    cc.d[2] = x[6]
    cc.d[4] = x[6]
    ### x_8 is d_2, which applies to layers 3 and 5
    cc.d[3] = x[7]
    cc.d[5] = x[7]
    
    ### now we have the new structure, update fresnel quantities
    cc.fresnel()
    ### now we have new emissivity, update thermal emission
    cc.thermal_emission()
    
    ### now we have new thermal emission, update luminous efficiency
    cc.luminous_efficiency()

    ### return luminous efficiency
    return cc.luminous_efficiency_val

### array of parameters
x = np.zeros(8)
### define lorentz parameters
c = 299792458
layer = 1
omega_0 = 2*np.pi*c/800e-9
gamma = 2*np.pi*c/7000e-9
omega_p = 2*np.pi*c/1000e-9

### Fill in x values
x[0] = omega_p
x[1] = omega_0
x[2] = gamma
x[3] = 10e-9
x[4] = 1.45
x[5] = 2.4
x[6] = 150e-9
x[7] = 100e-9

### create array of h values... 0.01*x should do it!
h = 0.01*x

test = update_multilayer(x)
print(test)


def BuildHessian(x0, h0):
    ### loop through elements i and j
    dim = len(x0)
    H = np.zeros((dim,dim))
    
    for i in range(0,dim):
        fxi = x0[i] + h0[i]
        bxi = x0[i] - h0[i]
        runi = 2*h0[i]
        
        for j in range(0, dim):
            fxj = x0[j] + h0[j]
            bxj = x0[j] - h0[j]
            runj = 2*h0[j]
            
            ### forward i forward j
            xpass = x0
            xpass[i] = fxi
            xpass[j] = fxj
            nff = update_multilayer(xpass)
            
            ### forward i backward j
            xpass[j] = bxj
            nfb = updated_multilayer(xpass)
            
            ### backward i backward j
            xpass[i] = bxi
            nbb = updated_multilayer(xpass)
            
            ### backward i forward j
            xpass[j] = fxj
            nbf = updated_multilayer(xpass)
            
            gf = (nff - nfb)/runj
            gb = (nbf - nbb)/runj
            
            H[i][j] = (gf - gb)/runi
            
    return H


    
    

0.00433523449908
